https://docs.celeryproject.org/en/stable/getting-started/next-steps.html#the-primitives

In [27]:
from celery import chain

In [28]:
!ls

basic1	basic3			 proj		test_celery
basic2	fastapi_rabbitmq_celery  simple_celery	Untitled.ipynb


In [29]:
#celery -A simple_celery worker -l info

In [30]:
from simple_celery.tasks import add, mul

In [31]:
res = add.delay(2,2)
res.get()

4

## Signatures

In [32]:
s1 = add.signature((2, 2), countdown=10)
res = s1.delay()
res.get()

4

## Partials

In [33]:
s1 = add.s(2)
res = s1.delay(2)
res.get()

4

## The Primitives: 
### group,  chain, chord,   map, starmap, chunks
These primitives are signature objects themselves, so they can be combined in any number of ways to compose complex work-flows.

## Groups
A group calls a list of tasks in parallel, and it returns a special result instance that lets you inspect the results as a group, and retrieve the return values in order.

In [34]:
from celery import group
from simple_celery.tasks import add, mul

In [35]:
group(add.s(i,i) for i in range(10))().get()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### Partial group

In [36]:
g = group(add.s(i) for i in range(10))
g(10).get()

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

## Chains
Tasks can be linked together so that after one task returns the other is called:

In [37]:
from celery import chain
from simple_celery.tasks import add, mul

In [38]:
# (4 + 4) * 8
chain(add.s(4, 4) | mul.s(8))().get()

64

or a partial chain:

In [39]:
# (? + 4) * 8
g = chain(add.s(4) | mul.s(8))
g(4).get()

64

Chains can also be written like this:

In [40]:
(add.s(4, 4) | mul.s(8))().get()

64

## Chords
A chord is a group with a callback:

In [41]:
from celery import chord
from simple_celery.tasks import add, mul, xsum

In [42]:
chord((add.s(i, i) for i in range(10)), xsum.s())().get()

90

A group chained to another task will be automatically converted to a chord:

In [43]:
(group(add.s(i, i) for i in range(10)) | xsum.s())().get()

90

## Routing
Celery supports all of the routing facilities provided by AMQP, but it also supports simple routing where messages are sent to named queues.
The task_routes setting enables you to route tasks by name and keep everything centralized in one location

## Remote Control
If you’re using RabbitMQ (AMQP), Redis, or Qpid as the broker then you can control and inspect the worker at runtime.

For example you can see what tasks the worker is currently working on:

$ celery -A proj inspect active

## Timezone
All times and dates, internally and in messages use the UTC timezone.
When the worker receives a message, for example with a countdown set it converts that UTC time to local time. If you wish to use a different timezone than the system timezone then you must configure that using the timezone setting:

app.conf.timezone = 'Europe/London'

## Optimization
The default configuration isn’t optimized for throughput.
If you have strict fair scheduling requirements, or want to optimize for throughput then you should read the Optimizing Guide.

## Canvas Guide
https://docs.celeryproject.org/en/stable/userguide/canvas.html#guide-canvas